# DIY ZHL

## References etc.

  * Mark Powell's "Deco for Divers" has all the background information in a succinct and easy to read form. The book should be available on interlibrary loan, ask your librarian.
  
  * Erik Baker's papers: "Decolessons", "Understanding M-values", "Clearing up the confusion about Deep Stops", are more technical and tend to omit much of the background info. PDFs are freely available on-line from several sources.
  
  * Buhlmann's "Decompression - Decompression sickness".
  
  * Several open-source implementations are out there, most notable Subsurface software, but you have to read C.
  
  * Plenty of other on-line sources, e.g. Stuart Morrison's "DIY Decompresion"


## Primer

### Dissolved gas

Gas is driven in and out of tissue by the difference in pressure: "outside" aka ambient $ P_{amb} $ &nbsp; and "inside" aka dissolved gas pressure, $ P_{t} $. The rate is assumed to be $ ln(2) $

**Partial pressure** refers to pressures of individual gases in the breathing mix: oxygen, nitrogen, etc.

### Tissue compartments

To simulate real tissue the model uses hypothetical "compartments". They are defined by the time $ T_{ h } $ it takes for the pressure difference (above) to get to -- since we're using log 2 -- half (or double, depending on whether the tissues in on- or off-gassing) its intial value. That time is callled ``half time``. It is usually given in minutes.

The body is modelled as a set of "tissue compartments" with different half-times. They have nothing to do with real tissues, but to illustrate the concept it is usually said that blood is "fast": gas gets in and out very quickly, whereas bone is very "slow".

### M-values

The greater the difference between $ P_{amb} $ &nbsp; and $ P_t $, the faster the gas flows. There is, however, such thing as too fast. To recreational divers "too fast" is only relevant on the way up.

The model defines "maximum allowed pressure difference" value, ``M-value`` for short. the model does not attempt to quantify the risk, it only works in binary "above the M-value" (bad) or "below the M-value" (OK) terms.

Haldane originally set his M-value as a fixed ratio (2:1). Workman replaced it with a function: a straight line $ M = \Delta M + M_0 $ with greater overpressure tolerated at greater depths.

## Water vapour etc. in alveolae

Breathing gas in respiratory passages is diluted with water vapour and expelled carbon dioxide. The fraction of water vapour and carbon dioxide is subtracted from the partial pressure of inert gas.

$
\begin{align}
P_{alv} = ( P_{amb} - P_{H_{2}O} + ( 1 - RQ ) / RQ * P_{CO_{2}} ) Q
\end{align}
$

where
  * $ P_{ CO_2 } $ is pressure of carbon dioxide, 40 mm Hg or 0.0534 bar,
  * $ RQ $ is respiratory quotient,
  * $ P_{ H_{2}O } $ is pressure of water vapour, 47 mm Hg or 0.0627 bar,
  * $ Q $ is the fraction of inert gas,
  * $ P_{amb} $ is ambient pressure.

$ RQ $ is a ratio of oxygen consumed to carbon dioxide produced in metabolism, it depends on nutrition and exertion and is in the range 0.7..1. Smaller number results in greater calculated gas loading. Buhlmann uses 1 whereas Shreiner uses 0.8 and USN uses 0.9. 

## Schreiner equation to calculate inert gas pressure in a tissue compartment


$
\begin{align}
P_{t}(t) = P_{alv0} + R * (t - 1 / k) - (P_{alv0} - P_{t0} - R / k) * e^{-k * t}
\end{align}
$

*where*

  * $ P_{t}(t) $ : partial pressure of inert gas in the tissue (bar) at time $ t $
  
  * $ P_{alv0} $ : initial alveolar partial pressure of the gas in the breathing mix (bar)
  
  * $ P_{t0} $ : initial partial pressure of the gas in the tissue (bar) (at time $ t0 $)
  
  * $ R $ is rate of change in inert gas pressure in the breathing mix, bar/min. This is rate of change in ambient pressure, $ \Delta P $ times fraction of inert gas. $ \Delta P $ is negative for ascent, positive for descent, zero for constant depth.

  * $ t $ : time of exposure in minutes

  * $ k $ : gas decay constant for a tissue compartment: $ k = \frac  {\ln(2)} {T_{h}} $

    * $ \quad T_{h} $ : gas "half-life" time for tissue compartment in minutes

  * $ e $ : 2.7182818284590452353602874713527 and counting

Note that for constant depth this reduces to a "short form" given in many sources, the code below just uses the full form with $ R = 0 $

## M-values

### Workman

Workman-style M-value $ M $ is defined in terms of $ \Delta M $ and $ M_{ 0 } $, where

  * $ M_{0} $ is the surfacing M-value and
  * $ \Delta M $ is the slope of M-value line: multiply by depth to get tha actual number at that depth.

$ M = \Delta M * Depth + M_{0} $


$ Tolerated Depth = ( P - M_{0} ) / \Delta M $

where Depth is measured as pressure from sea level and Tolerated Depth is how high you can ascend without exceeding the M-value.

### Buhlmann

In Buhlmann model the M-values are expresed in terms of two coeficients: $ a $ and $ b $. It is also using absolute pressure for use when diving at altitude, and comes with separate numbers for different inert gases. Buhlmann's model calls "Tolerated Depth" an "ascent ceiling".

### Conversion

  * Workman to Buhlmann:
  
   * $ a = M_{0} - \Delta M * P_{ \text{ amb. (surface at sea level) }} $ 
   
   * $ b = 1 / \Delta M $  

  * Buhlmann to Workman:

   * $ \Delta M = 1 / b $
   
   * $ M_{0} = a + \frac { P_{ \text{ amb. (surface at sea level) }}} { b } $
   
**Note** that this only really works with $ \Delta M $ greater than 1. See Erik Baker's "M-values" for the why.
   

## No-stop time

### by (variant of) Schreiner equation

This needs Workman-style M-values:

$
\begin{align}
P_{no-deco} \ = ( M_{0} - P_{alv0} - R * (t_{asc} - 1/k) ) * e ^ {k * t_{asc}} \ + P_{alv0} - R / k
\end{align}
$
  * $ P_{no-deco} $ &nbsp; is the pressure at which ascent must be started (bar),
  * $ M_{0} $ is the surfacing M-value,
  * $ P_{alv0} $ is alveolar partial pressure (bar),
  * $ t_{asc} $ is the time needed for ascent (minutes): depth divided by speed.
  * $ R, k, e $ are the same as in Schreiner's equation above.
  
You have to figure out how long it'll take to get from the current inspired pressure $ P_{t} $ to $ P_{no-deco} $ &nbsp; : run `schreiner()` in a loop adding a minute of exposure on each iteration until you hit $ P_{no-deco} $ &nbsp; (or do a fancy binary search from, say, 99 minutes down, etc.)

### by Buhlmann equation

Buhlmann equation calculates Tolerated Depth in absolute pressure and calls it "safe ascent ceiling":

$
\begin{align}
P_{amb.tol.} = ( P_{t} - a ) * b
\end{align}
$

where
  * $ P_{amb.tol.} $ &nbsp; is the ascent ceiling,
  * $ P_{t} $ is current inert gas pressure in the tisue, and
  * $ a $ and $ b $ are Buhlmann's coefficients.

When the calculated $ P_{amb.tol.} $ &nbsp; is above 1, i.e. under the water surface (assuming surface pressure is 1 bar of course), that means we have a deco ceiling and must stop and off-gas there.

Buhlmann equation modified by Erik Baker's Gradient Factors is

$
\begin{align}
P_{amb.tol.} = ( P_{t} - a * GF ) / ( GF / b + 1.0 - GF )
\end{align}
$

where $ GF $ is the "current" gradient factor. 

(Without the gradient factor: $ GF = 1 $, that reduces to $ ( P_{t} - a * 1 ) / ( 1/b + 1 - 1 ) => ( P_{t} - a ) / ( 1/b ) =>  ( P_{t} - a ) * b $: Buhlmann's original.)

### You can calculate no-stop time
by running `shreiner()` and `buhlmann()` in a loop, adding a minute of exposure at each iteration, until the latter produces a ceiling. **Note** however, that neither method factors in off-gassing that will take place during ascent.

### Helium

With multiple inert gases, each has to be tracked separately by Schreiner equation.

For Buhlmann's formula you add up the partial pressures and proportionally adjust the $ a $ and $ b $:

$ P_{t} = P_{iN_2} + P_{iHe} $

$ a = (a_{N_2} * P_{N_2} + a_{He} * P_{He}) / P_{t} $

$ b = (b_{N_2} * P_{N_2} + b_{He} * P_{He}) / P_{t} $



## Decompression stop

... is ascent ceiling rounded to next 3 metres down.

The best I could find, in Buhlmann's "Decompression", is in open water it's hard to maintain stops at smaller intervals. Buhlmann suggests using smaller intervals or even "ride the M-value line" continuously for in-chamber decompression there. (**Note** this assumes no gas switching.)

Once you're at deco stop, run `schreiner()` and `buhlmann()` in a loop adding a minute on each iteration, as usual, until your next ceiling is at your next stop: 3 metres above you current depth if using the 3 m spacing.

## Tissue compartments

Tables are coded as python dicts: keys for each compartment are ``t``, ``a``, and ``b`` for half-time and the coefficients resp. E.g. ZH-L12's $ b $ coefficient for nitrogen in TC 3 is ``ZHL12N[3]["b"]``


### ZH-L16

**Note** that you have two options for the 1st (fastest) tissue compartment: 4 minutes or 5. Pick one. The 5-minute TC is usually called "1b" and is keyed as "1.1" in the table below.

The ''a'' key in the nitrogen tables is a dict itself with three sub-keys: ''A'', ''B'', adn ''C'' for the three versions of ZH-L16 resp. Helium numbers seem to be unobtainable for ''A'' or ''C'', nor for the 4-minute tissue compartment. I.e. if you want to use this for trimix, ZH-L16B with 5-minute TC is your only option.

M-values for "-A" were calculated as

$
\begin{align}
a = 2 * T_{h} ^ {- \frac{1}{3}}
\end{align}
$

$
\begin{align}
b = 1.005 - T_{h} ^ {- \frac{1}{2}}
\end{align}
$


  * $ T_{h} $ is tissue compartment half-time


### Workman

Not useful as is, would need to be recalculated for $ \Delta M $ instead of $ M $ first, then converted to Buhlmann's `a` and `b` coefficients.

### DSAT

Convert to Buhlmann's `a` and `b` using $ \Delta M = 1 $

## Code

Run the cell below to load python code (assuming you have the `diyzhl.py` in the same directory as the notebook file).

The functions are written to closely follow the above formulae. That includes calculating constants on the fly, like e.g. $ K $ or $ P_{alv} $ &nbsp; -- you'd normally use pre-calculated numbers for your chosen parameters.

About the only added features are 
 * some of them save intermediate results in variables: mainly because I wanted to play with rounding at different steps. Note that although I round the numbers to 4 decimal digits, they're probably only meaningful to 2 at best.
 
 * Typecasts to `float` are there so that passing in not-a-number will throw an exception early.      
 
 * Similarly, `asserts` are used to catch division by zero early.

In [1]:
# %load diyzhl.py
#!/usr/bin/python -u
#
# (K) Copy Rites Reversed: reuse what you like (but give credit)
#
# Credits:
#
# Mark Powell's "Deco for Divers"
# Erik Baker's papers: "Decolessons", "Understanding M-values", and "Clearing up the confusion about Deep Stops" in particular
# Buhlmann's "Decompression - Decompression Sickness", English edition
# Several open-source implementations, most notably Subsurface software (and people, Robert in particular)
# Plenty of other on-line sources, e.g. Stuart Morrison's "DIY Decompresion"
#
# The goal here is "by the book" implementation to use for learning this stuff.
# There's a lot of things that can be done differently, more efficeint, and so on... and
#  that's not what this code is for.
#

import sys
import math

# ZH-L12 from "Decompression"
#
ZHL12N = {
    1 :  { "t" : 2.65,  "a" : 2.2,   "b" : 0.82 },
    2 :  { "t" : 7.94,  "a" : 1.5,   "b" : 0.82 },
    3 :  { "t" : 12.2,  "a" : 1.08,  "b" : 0.825 },
    4 :  { "t" : 18.5,  "a" : 0.9,   "b" : 0.835 },
    5 :  { "t" : 26.5,  "a" : 0.75,  "b" : 0.845 },
    6 :  { "t" : 37.0,  "a" : 0.58,  "b" : 0.86 },
    7 :  { "t" : 53.0,  "a" : 0.47,  "b" : 0.87 },
    8 :  { "t" : 79.0,  "a" : 0.45,  "b" : 0.89 },
    9 :  { "t" : 114.0, "a" : 0.45,  "b" : 0.89 },
    10 : { "t" : 146.0, "a" : 0.455, "b" : 0.934 },
    11 : { "t" : 185.0, "a" : 0.455, "b" : 0.934 },
    12 : { "t" : 238.0, "a" : 0.38,  "b" : 0.944 },
    13 : { "t" : 304.0, "a" : 0.255, "b" : 0.962 },
    14 : { "t" : 397.0, "a" : 0.255, "b" : 0.962 },
    15 : { "t" : 503.0, "a" : 0.255, "b" : 0.962 },
    16 : { "t" : 635.0, "a" : 0.255, "b" : 0.962 }
}

ZHL12He = {
    1 :  { "t" : 1.0,   "a" : 2.2,   "b" : 0.82 },
    2 :  { "t" : 3.0,   "a" : 1.5,   "b" : 0.82 },
    3 :  { "t" : 4.6,   "a" : 1.08,  "b" : 0.825 },
    4 :  { "t" : 7.0,   "a" : 0.9,   "b" : 0.835 },
    5 :  { "t" : 10.0,  "a" : 0.75,  "b" : 0.845 },
    6 :  { "t" : 14.0,  "a" : 0.58,  "b" : 0.86 },
    7 :  { "t" : 20.0,  "a" : 0.47,  "b" : 0.87 },
    8 :  { "t" : 30.0,  "a" : 0.45,  "b" : 0.89 },
    9 :  { "t" : 43.0,  "a" : 0.45,  "b" : 0.89 },
    10 : { "t" : 55.0,  "a" : 0.515, "b" : 0.926 },
    11 : { "t" : 70.0,  "a" : 0.515, "b" : 0.926 },
    12 : { "t" : 90.0,  "a" : 0.515, "b" : 0.926 },
    13 : { "t" : 115.0, "a" : 0.515, "b" : 0.926 },
    14 : { "t" : 150.0, "a" : 0.515, "b" : 0.926 },
    15 : { "t" : 190.0, "a" : 0.515, "b" : 0.926 },
    16 : { "t" : 240.0, "a" : 0.515, "b" : 0.926 },
}

# ZH_L16: several sources incl. a photo of a page from Tauchmedizin @
# http://www.nigelhewitt.co.uk/stuff/aab.jpg
# It appears nobody has Helium numbers for "-A" and "-C", nor for the 4-minute TC
# 5-minute TC is keyed as 1.1
#
ZHL16N = {
    1   : { "t" : 4.0,   "b" : 0.505,  "a" : { "A" : 1.2599, "B" : 1.2599, "C" : 1.2599 } },
    1.1 : { "t" : 5.0,   "b" : 0.5578, "a" : { "A" : 1.1696, "B" : 1.1696, "C" : 1.1696 } },
    2   : { "t" : 8.0,   "b" : 0.6514, "a" : { "A" : 1.0,    "B" : 1.0,    "C" : 1.0 } },
    3   : { "t" : 12.5,  "b" : 0.7222, "a" : { "A" : 0.8618, "B" : 0.8618, "C" : 0.8618 } },
    4   : { "t" : 18.5,  "b" : 0.7825, "a" : { "A" : 0.7562, "B" : 0.7562, "C" : 0.7562 } },
    5   : { "t" : 27.0,  "b" : 0.8126, "a" : { "A" : 0.6667, "B" : 0.6667, "C" : 0.62 } },
    6   : { "t" : 38.3,  "b" : 0.8434, "a" : { "A" : 0.5933, "B" : 0.56,   "C" : 0.5043 } },
    7   : { "t" : 54.3,  "b" : 0.8693, "a" : { "A" : 0.5282, "B" : 0.4947, "C" : 0.441 } },
    8   : { "t" : 77.0,  "b" : 0.891,  "a" : { "A" : 0.4701, "B" : 0.45,   "C" : 0.4 } },
    9   : { "t" : 109.0, "b" : 0.9092, "a" : { "A" : 0.4187, "B" : 0.4187, "C" : 0.375 } },
    10  : { "t" : 146.0, "b" : 0.9222, "a" : { "A" : 0.3798, "B" : 0.3798, "C" : 0.35 } },
    11  : { "t" : 187.0, "b" : 0.9319, "a" : { "A" : 0.3497, "B" : 0.3497, "C" : 0.3295 } },
    12  : { "t" : 239.0, "b" : 0.9403, "a" : { "A" : 0.3223, "B" : 0.3223, "C" : 0.3065 } },
    13  : { "t" : 305.0, "b" : 0.9477, "a" : { "A" : 0.2971, "B" : 0.285,  "C" : 0.2835 } },
    14  : { "t" : 390.0, "b" : 0.9544, "a" : { "A" : 0.2737, "B" : 0.2737, "C" : 0.261 } },
    15  : { "t" : 498.0, "b" : 0.9602, "a" : { "A" : 0.2523, "B" : 0.2523, "C" : 0.248 } },
    16  : { "t" : 635.0, "b" : 0.9653, "a" : { "A" : 0.2327, "B" : 0.2327, "C" : 0.2327 } }
}

ZHL16He = {
    1.1 : { "t" : 1.88,   "a" : { "B" : 1.6189 }, "b" : 0.477 },
    2 :   { "t" : 3.02,   "a" : { "B" : 1.383 },  "b" : 0.5747 },
    3 :   { "t" : 4.72,   "a" : { "B" : 1.1919 }, "b" : 0.6527 },
    4 :   { "t" : 6.99,   "a" : { "B" : 1.0458 }, "b" : 0.7223 },
    5 :   { "t" : 10.21,  "a" : { "B" : 0.922 },  "b" : 0.7582 },
    6 :   { "t" : 14.48,  "a" : { "B" : 0.8205 }, "b" : 0.7957 },
    7 :   { "t" : 20.53,  "a" : { "B" : 0.7305 }, "b" : 0.8279 },
    8 :   { "t" : 29.11,  "a" : { "B" : 0.6502 }, "b" : 0.8553 },
    9 :   { "t" : 41.20,  "a" : { "B" : 0.595 },  "b" : 0.8757 },
    10 :  { "t" : 55.19,  "a" : { "B" : 0.5545 }, "b" : 0.8903 },
    11 :  { "t" : 70.69,  "a" : { "B" : 0.5333 }, "b" : 0.8997 },
    12 :  { "t" : 90.34,  "a" : { "B" : 0.5189 }, "b" : 0.9073 },
    13 :  { "t" : 115.29, "a" : { "B" : 0.5181 }, "b" : 0.9122 },
    14 :  { "t" : 147.42, "a" : { "B" : 0.5176 }, "b" : 0.9171 },
    15 :  { "t" : 188.24, "a" : { "B" : 0.5172 }, "b" : 0.9217 },
    16 :  { "t" : 240.03, "a" : { "B" : 0.5119 }, "b" : 0.9267 }
}

# From Deco for Divers
# Do not use: 
#  need to convert to bar and delta-M to use,
#  M0 is in msw and M calculated as Delta M * Depth + M0,
#  Depth is unknown
#
WORKMAN = {
    1 : { "t" : 5.0, "M0" : 31.5, "M" : 1.8 },
    2 : { "t" : 10.0, "M0" : 26.8, "M" : 1.6 },
    3 : { "t" : 20.0, "M0" : 21.9, "M" : 1.5 },
    4 : { "t" : 40.0, "M0" : 17.0, "M" : 1.4 },
    5 : { "t" : 80.0, "M0" : 16.4, "M" : 1.3 },
    6 : { "t" : 120.0, "M0" : 15.8, "M" : 1.2 },
    7 : { "t" : 160.0, "M0" : 15.5, "M" : 1.15 },
    8 : { "t" : 200.0, "M0" : 15.5, "M" : 1.1 },
    9 : { "t" : 240.0, "M0" : 15.2, "M" : 1.1 }
}

# Also from Deco for Divers
#
# Since DSAT's primary concern is no-stop diving, it only uses M0
#  -- there is no Delta M i.e. Delta M = 1
# values are in msw
#
# Conveert to Buhlmann with
#  m_w2b( M0 = NNN / 10, dM = 1, P = 1 )
# and run ZHL with DSAT compartments and M-values
#
DSAT = {
    1 : { "t" : 5.0, "M0" : 30.42 },
    2 : { "t" : 10.0, "M0" : 25.37 },
    3 : { "t" : 20.0, "M0" : 20.54 },
    4 : { "t" : 30.0, "M0" : 18.34 },
    5 : { "t" : 40.0, "M0" : 17.11 },
    6 : { "t" : 60.0, "M0" : 15.79 },
    7 : { "t" : 80.0, "M0" : 15.11 },
    8 : { "t" : 100.0, "M0" : 14.69 },
    9 : { "t" : 120.0, "M0" : 14.41 },
    10 : { "t" : 160.0, "M0" : 14.06 },
    11 : { "t" : 200.0, "M0" : 13.84 },
    12 : { "t" : 240.0, "M0" : 13.69 },
    13 : { "t" : 360.0, "M0" : 13.45 },
    14 : { "t" : 480.0, "M0" : 13.33 }
}

# return alveolar inert gas pressure
# with P amb = 1 bar, fraction of inert gas = 0.79, and RQ = 0.9
# this should return 0.79 - 0.0567 = 0.7451 or 0.7452 dep. on where you round it
#
def palv( Pamb = 1, Q = 0.79, RQ = 0.9 ) :
    assert float( RQ ) != 0.0
    vw = float( Pamb ) - 0.0627 + (1.0 - float( RQ )) / float( RQ ) * 0.0534
    return round( vw * float( Q ), 4 )

# return k: constant for tissue compartment (min^-1)
# Th : tissue compartment half-time in minutes
# for 5-minute compartment it's 0.8452
#
def kay( Th = 5 ) :
    assert float( Th ) > 0.0
    return round( math.log( 2 ) / float( Th ), 4 )

# return rate of pressure change in bar/min
# d0 : start pressure, bar
# dt : end pressure, bar
# t : time, min
# Q : fraction of inert gas (same Q as in palv()
#
def arr( d0 = 1.0, dt = 1.0, t = 1, Q = 0.79 ) :
    assert float( t ) > 0.0
    dP = (float( dt ) - float( d0 )) / float( t )
    rc = dP * float( Q )
    return round( rc, 4 )

# Schreiner equation
# Palv + R * (t - 1/k) - (Palv - Pi - R/k) * e^(-k * t)
#
# returns pressure in tissue compartment after time t at depth Pa & dP
#
# Pi: initial pressure of inert gas in tissue (bar)
# Palv: initial pressure of inert gas in the lungs (bar, output of palv())
# t: time (minutes)\n",
# R: rate of pressure change (output of arr()),
# k: gas decay constant (output of kay()
#
# (Intermediate variables b/c I was playing with rounding)
#

def schreiner( Pi = 0.7451, Palv = 0.7451, t = 1, R = 0, k = 0.1386, verbose = False ) :

    assert float( k ) != 0.0
    x1 = float( R ) * (float( t ) - 1.0 / float( k ))
    x2 = float( Palv ) - float( Pi ) - float( R ) / float( k )
    x3 = math.e ** (float( -k ) * float( t ))
    rc = round( float( Palv ) + x1 - x2 * x3, 4 )
    if verbose : sys.stdout.write( "x1: %f, x2: %f, x3: %f, rc: %f\n" % (x1, x2, x3, rc,) )
    return round( rc, 4 )

# M-value: workman to buhlmann
# P is ambient pressure in bar
# returns pair ( a, b )
#
# TODO: add GF
#
def m_w2b( M0 = 2.9624, dM = 1.7928, P = 1 ) :
    assert float( dM ) >= 1.0
    a = float( M0 ) - float( dM ) * float( P )
    b = 1.0 / float( dM )
    return (round( a, 4 ), round( b, 4 ))

# M-value: buhlmann to workman
# returns pair ( M0, dM )
#
def m_b2w( a = 1.1696, b = 0.5578, P = 1 ) :
    assert float( b ) > 0.0
    M0 = float( a ) + float( P ) / float( b )
    dM = 1.0 / float( b )
    return (round( M0, 4 ), round( dM, 4 ))

# no-stop time by Schreiner
#
# Palv: initial pressure of inert gas in the lungs (bar, output of palv())
# t: time (minutes)\n",
# R: rate of pressure change (output of arr()),
# k: gas decay constant (output of kay()
#  -- same as schreiner()
# M0: surfacing M-value as per Workman
#
def ndl( Palv = 0.7451, M0 = 2.9624, t = 0, R = 0, k = 0.1386, verbose = False ) :

# (M0 - Palv - R * (t - 1/k)) * math.e ** (k * t) + Palv - R / k
    assert float( k ) != 0.0
    x1 = float( M0 ) - float( Palv ) - float( R ) * (float( t ) - 1.0 / float( k ))
    x2 = math.e ** (float( k ) * float( t ))
    rc = x1 * x2 + float( Palv ) - float( R ) / float( k )
    if verbose : sys.stdout.write( "x1: %f, x2: %f, rc: %f\n" % (x1, x2, rc,) )
    return round( rc, 4 )

# Buhlman formula with GF and Helium
# returns safe ascent ceiling
#
# Pn is N2 pressure in tissue compartment
# Phe is He pressure in tissue compartment
# an is N2 a coefficient
# bn is N2 b coefficient
# ahe is He a coefficient
# bhe is He b coefficient
# gf is current gradient factor
#
def buhlmann( Pn, an, bn, Phe = 0, ahe = 0, bhe = 0, gf = 1 ) :

    P = float( Pn ) + float( Phe )
    assert float( P ) != 0.0
    a = (float( an ) * float( Pn ) + float( ahe ) * float( Phe )) / P
    b = (float( bn ) * float( Pn ) + float( bhe ) * float( Phe )) / P
    num = float( P ) - float( a ) * float( gf )
    den = float( gf ) / float( b ) + 1.0 - float( gf )
    assert den != 0.0
    rc = num / den
    return round( rc, 4 )

# eof
#
